In [3]:
import pandas as pd
cases_train= pd.read_csv('data/cases_2021_train_processed_2.csv')
cases_test= pd.read_csv('data/cases_2021_test_processed_unlabelled_2 .csv')


In [5]:
cases_test.isna().sum() 

age                          0
sex                          0
province                  3621
country                      0
latitude                     0
longitude                    0
date_confirmation            0
chronic_disease_binary       0
Confirmed                    0
Deaths                       0
Recovered                    0
Active                       0
Incident_Rate                0
Case_Fatality_Ratio          0
outcome_group                0
dtype: int64

In [28]:
print(cases_train.groupby('outcome_group').size())
cases = cases_train.drop(cases_train[(cases_train["province"] == cases_train["country"]) & (cases_train["outcome_group"] == "hospitalized")].index)
print(cases.groupby('outcome_group').size())

outcome_group
deceased             997
hospitalized       13241
nonhospitalized     2974
dtype: int64
outcome_group
deceased             997
hospitalized       13161
nonhospitalized     2974
dtype: int64


In [34]:
test = cases_train[(cases_train["province"] == cases_train["country"]) ]
test.groupby('outcome_group').size()

outcome_group
deceased            689
hospitalized         80
nonhospitalized    2852
dtype: int64

In [36]:
cases_train= pd.read_csv('data/cases_2021_test_processed_unlabelled_2.csv')
cases_train.isna().sum()

age                       0
sex                       0
province                  0
country                   0
latitude                  0
longitude                 0
date_confirmation         0
chronic_disease_binary    0
Confirmed                 0
Deaths                    0
Recovered                 0
Active                    0
Incident_Rate             0
Case_Fatality_Ratio       0
dtype: int64